In [ ]:
import torch
import torchmetrics
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    accuracy_score, precision_score, recall_score, f1_score
)
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch import nn, optim
import torch.nn.init as init